In [1]:
###magic function which plot the figures automaticaly and keeps autoreload for updated modules 
%matplotlib inline
%reload_ext autoreload
%autoreload 2

From here on, I will try some hyper linke to organize my notebook

1.[read csv files](#read_csv)

In [4]:
###corresponding module in fastai for structure data
from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)

##specify the main folder address
PATH='D:\deeplearning_data\\rossmann\\'

ModuleNotFoundError: No module named 'fastai'

Here is a function which used to concate the csv file interested

In [5]:
def concat_csvs(dirname):        ##pass in the name of directory
    path = f'{PATH}{dirname}'    ##concatenate it to get file name 
    filenames=glob(f"{PATH}*.csv")  ##get all the csv file

    wrote_header = False                            
    with open(f"{path}.csv","w") as outputfile:      ##with function will automaticly clsoe the open file
        for filename in filenames:                  
            name = filename.split(".")[0]            ## get one name of file
            with open(filename) as f:                ## open it and read line information
                line = f.readline()                    
                if not wrote_header:                 
                    wrote_header = True              ## use wrote_header as a flag ()
                    outputfile.write("file,"+line)   ## 
                for line in f:
                     outputfile.write(name + "," + line)
                outputfile.write("\n")
#concat_csvs('googletrend')

In [ ]:
table_names = ['train', 'store', 'store_states', 'state_names', 
               'googletrend', 'weather', 'test']  ##list of name interested

<a id="read_csv"></a>
<font color=Black size=5> The next step is to read in the csv files as pandas data frame </font>

In [1]:
tables = [pd.read_csv(f'{PATH}{fname}.csv', low_memory=False) for fname in table_names] ##read them in as csv file

NameError: name 'table_names' is not defined

In [ ]:
for t in tables: display(t.head()) ## see the data

In [ ]:
for t in tables: display(t.de) ## summary of data which will only focus on numerical values

In [ ]:
train, store, store_states, state_names, googletrend, weather, test = tables  ##separate them into sub dataframe

In [ ]:
train.StateHoliday.unique()   ##check the number of unique values inside of specific columns

In [ ]:
train.StateHoliday = train.StateHoliday!='0'
test.StateHoliday = test.StateHoliday!='0'   ##convert boolean variables

In [2]:
def join_df(left, right, left_on, right_on=None, suffix='_y'):   ### this is a function which take a use of pandas.dataframe.merge
    if right_on is None: right_on = left_on                      ### it join two tables on "left_on"= "right_on" and add suffix 
    return left.merge(right, how='left', left_on=left_on, right_on=right_on,   ##to columns duplicated
                      suffixes=("", suffix))
##EXAMPLE :weather = join_df(weather, state_names, "file", "StateName")

In [ ]:
##separate the information into small subset[attributes] 
googletrend['State'] = googletrend.file.str.split('_', expand=True)[2] 
googletrend['Date'] = googletrend.week.str.split(' - ', expand=True)[0]
##make sure that attributes has same expression in different tables
googletrend.loc[googletrend.State=='NI', "State"] = 'HB,NI'

In [ ]:
##generate data-time attribute construction , whether it is holiday and so on
add_datepart(weather, "Date", drop=False)
add_datepart(googletrend, "Date", drop=False)
add_datepart(train, "Date", drop=False)
add_datepart(test, "Date", drop=False)

In [ ]:
## the external data 
trend_de = googletrend[googletrend.file == 'Rossmann_DE']

In [ ]:
##1. the merge function can also handle multiple criteria
##2.***** after left outer join, check to see if there is null variables * * * 
joined = join_df(joined, googletrend, ["State","Year", "Week"])
joined_test = join_df(joined_test, googletrend, ["State","Year", "Week"])
len(joined[joined.trend.isnull()]),len(joined_test[joined_test.trend.isnull()])

In [ ]:
##drop those duplicated columns
for df in (joined, joined_test):
    for c in df.columns:
        if c.endswith('_y'):
            if c in df.columns: df.drop(c, inplace=True, axis=1)

In [ ]:
##fill up NAs
for df in (joined,joined_test):
    df['CompetitionOpenSinceYear'] = df.CompetitionOpenSinceYear.fillna(1900).astype(np.int32)
    df['CompetitionOpenSinceMonth'] = df.CompetitionOpenSinceMonth.fillna(1).astype(np.int32)
    df['Promo2SinceYear'] = df.Promo2SinceYear.fillna(1900).astype(np.int32)
    df['Promo2SinceWeek'] = df.Promo2SinceWeek.fillna(1).astype(np.int32)

In [ ]:
##organize to time series variable
for df in (joined,joined_test):
    df["CompetitionOpenSince"] = pd.to_datetime(dict(year=df.CompetitionOpenSinceYear, 
                                                     month=df.CompetitionOpenSinceMonth, day=15))
    df["CompetitionDaysOpen"] = df.Date.subtract(df.CompetitionOpenSince).dt.days

In [ ]:
##deal with outlier 
for df in (joined,joined_test):
    df.loc[df.CompetitionDaysOpen<0, "CompetitionDaysOpen"] = 0
    df.loc[df.CompetitionOpenSinceYear<1990, "CompetitionDaysOpen"] = 0
for df in (joined,joined_test):
    df["CompetitionMonthsOpen"] = df["CompetitionDaysOpen"]//30
    df.loc[df.CompetitionMonthsOpen>24, "CompetitionMonthsOpen"] = 24
joined.CompetitionMonthsOpen.unique()

In [ ]:
##to_feather a fast way to store ram data 
joined.to_feather(f'{PATH}joined')
joined_test.to_feather(f'{PATH}joined_test')

##use read_feather to load
df = pd.read_feather(f'{PATH}df')

In [ ]:
##track the time until next interesting event happens
def get_elapsed(fld, pre):
    day1 = np.timedelta64(1, 'D')
    last_date = np.datetime64()
    last_store = 0
    res = []

    for s,v,d in zip(df.Store.values,df[fld].values, df.Date.values):
        if s != last_store:
            last_date = np.datetime64()
            last_store = s
        if v: last_date = d
        res.append(((d-last_date).astype('timedelta64[D]') / day1))
    df[pre+fld] = res
fld = 'SchoolHoliday'
df = df.sort_values(['Store', 'Date'])
get_elapsed(fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')

In [ ]:
##changing index of the data frame
df = df.set_index("Date")

In [ ]:
##fill NAs
columns = ['SchoolHoliday', 'StateHoliday', 'Promo']
for o in ['Before', 'After']:
    for p in columns:
        a = o+p
        df[a] = df[a].fillna(0).astype(int)

<font color=Black size=3>
Here we're sorting by date (sort_index()) and counting the number of events of interest (sum()) defined in columns in the following week (rolling()), grouped by Store (groupby()). We do the same in the opposite direction.
more explaination in the figure
</font>

In [ ]:
bwd ==bwd = df[['Store']+columns].sort_index().groupby("Store").rolling(7, min_periods=1).sum()
fwd = df[['Store']+columns].sort_index(ascending=False
                                      ).groupby("Store").rolling(7, min_periods=1).sum()
#them get away with duplicated columns
bwd.drop('Store',1,inplace=True)
bwd.reset_index(inplace=True)
fwd.drop('Store',1,inplace=True)
fwd.reset_index(inplace=True)
##reset the index back
joined.reset_index(inplace=True)
joined_test.reset_index(inplace=True)

<font color=Black size=3>
Separate them according to their property(char/num)
</font>

In [ ]:

cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen',
    'Promo2Weeks', 'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear',
    'State', 'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_fw', 'StateHoliday_bw',
    'SchoolHoliday_fw', 'SchoolHoliday_bw']

contin_vars = ['CompetitionDistance', 'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
   'AfterStateHoliday', 'BeforeStateHoliday', 'Promo', 'SchoolHoliday']


In [ ]:
##grab a subset of the data
dep = 'Sales'
joined = joined[cat_vars+contin_vars+[dep, 'Date']].copy()

joined_test[dep] = 0
joined_test = joined_test[cat_vars+contin_vars+[dep, 'Date', 'Id']].copy()

In [ ]:
##convert char into category type ; fill NAs in num
for v in cat_vars: joined[v] = joined[v].astype('category').cat.as_ordered()
for v in contin_vars:
    joined[v] = joined[v].fillna(0).astype('float32')
    joined_test[v] = joined_test[v].fillna(0).astype('float32')

In [ ]:
##separate the data and labels. 
df, y, nas, mapper = proc_df(joined_samp, 'Sales', do_scale=True)
yl = np.log(y)
df_test, _, nas, mapper = proc_df(joined_test, 'Sales', do_scale=True, skip_flds=['Id'],
                                  mapper=mapper, na_dict=nas)

In [ ]:
##separate validation set
train_ratio = 0.75
# train_ratio = 0.9
train_size = int(samp_size * train_ratio); train_size
val_idx = list(range(train_size, len(df)))
# another way
val_idx = np.flatnonzero(
    (df.index<=datetime.datetime(2014,9,17)) & (df.index>=datetime.datetime(2014,8,1)))

In [ ]:
##define evaluation matrix
def inv_y(a): return np.exp(a)

def exp_rmspe(y_pred, targ):
    targ = inv_y(targ)
    pct_var = (targ - inv_y(y_pred))/targ
    return math.sqrt((pct_var**2).mean())

max_log_y = np.max(yl)
y_range = (0, max_log_y*1.2)

In [ ]:
##feed the data to pytorch tensor
md = ColumnarModelData.from_data_frame(PATH, val_idx, df, yl.astype(np.float32), cat_flds=cat_vars, bs=128,
                                       test_df=df_test)

In [ ]:
##build embedding matrix
cat_sz = [(c, len(joined_samp[c].cat.categories)+1) for c in cat_vars]
emb_szs = [(c, min(50, (c+1)//2)) for _,c in cat_sz]
m = md.get_learner(emb_szs, len(df.columns)-len(cat_vars),
                   0.04, 1, [1000,500], [0.001,0.01], y_range=y_range)
lr = 1e-3

In [ ]:
##fit models
m.fit(lr, 3, metrics=[exp_rmspe])